<span style="color:#78AE7E">

Aline's comments are in green. :)

<span>

# Stabilities and accuracy in time. 

Lets consider same setup as in [ex_3a](https://github.com/AST-Course/AST5110/blob/main/ex_3a.ipynb) Burgers’ equation, i.e.,

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0   \tag{1}$$ 

for the domain $x \in (x_0, x_f)$ with $x_0 = −1.4$, $x_f = 2.0$ with initial condition:

$$u(x,t=0) = A\left[\tanh\left(\frac{x+x_c}{W}\right)-\tanh\left(\frac{x-x_c}{W}\right)\right]   \tag{2}$$

whereby $A = 0.02$ , $x_c = 0.70$, $W = 0.1$. Let the solution evolve until time $t_f = 100$. Explain in physical (or mathematical) terms the solution you get.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
import matplotlib as mpl
mpl.rc('lines', linewidth=2)

from nm_lib import nm_lib as nm

def u_0(x: np.ndarray, t: float = 0) -> np.ndarray:
    r"""
    Initial condition for the advection equation.

    Parameters
    ----------
    x : `array`
        the x-axis.
    t : `float`
        the time.
    
    Returns
    -------
    `array`
        the initial condition.
    """
    return A * ( np.tanh((x + x_c) / W) - np.tanh((x - x_c) / W) )

In [2]:
# Setup for animations

from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def init(): 
    """Initialize animation"""
    axes.plot(xx,unnt[:,0])

def animate(i):
    """Animate the solution with dynamic axis limits"""
    axes.clear()
    axes.plot(xx,unnt[:,i])
    axes.set_title('t=%.2f'%t[i])

In [3]:
A = 0.02
x_c = 0.7 
W = 0.1

However, let's now implement a new time-step method. 

Add the following method to your library and run the previous simulation. 

$$u^{n+1}_j = \frac{1}{2}(u^n_{j+1} + u^n_{j-1}) - \frac{v \Delta t}{2 \Delta x}(u^n_{j+1} - u^n_{j-1})$$

Apply von Neumann stability analysis to the expression above. The stability condition $|\xi(k)|^2 \leq 1$ leads to the famous Courant-Friedrichs-Lewy (CFL) stability. 

<span style="color:#78AE7E">

<!-- ### Solution -->

From [AST5110_wiki/Stability](https://github.com/AST-Course/AST5110/wiki/Stability) we have that the eigenmodes are on the form $u_j^n = \xi^n e^{ikj\Delta x}$. When we insert this into the time-step method above, we get that

\begin{align}
    \xi^{n+1} e^{ikj\Delta x} &= \frac{1}{2} \left( \xi^n e^{ik(j+1)\Delta x} + \xi^n e^{ik(j-1)\Delta x} \right) - \frac{v \Delta t}{2 \Delta x} \left( \xi^n e^{ik(j+1)\Delta x} - \xi^n e^{ik(j-1)\Delta x} \right) \nonumber \\
    &= \frac{1}{2} \xi^n \left( e^{ikj\Delta x} (e^{ik\Delta x} + e^{-ik\Delta x}) \right) - \frac{v\Delta t}{2\Delta x} \xi^n \left( e^{ikj\Delta x} (e^{ik\Delta x} - e^{-ik\Delta x}) \right). \nonumber 
\end{align}

<span>

<span style="color:#78AE7E">

Dividing both sides by $\xi^n e^{ikj\Delta x}$ yields

\begin{align}
    \xi = \frac{1}{2} \left( e^{ik\Delta x} + e^{-ik\Delta x} \right) - \frac{v\Delta t}{2\Delta x} \left( e^{ik\Delta x} - e^{-ik\Delta x} \right). \nonumber
\end{align}

The Euler formula lets us rephrase this and simplify the equation further.

\begin{align}
    \xi = &\frac{1}{2} \left( \cos(k\Delta x) + i\sin(k\Delta x) + \cos(k\Delta x) - i\sin(k\Delta x)  \right) \nonumber \\
    &- \frac{v\Delta t}{2\Delta x} \left( \cos(k\Delta x) + i\sin(k\Delta x) - \cos(k\Delta x) + i\sin(k\Delta x) \right) \nonumber \\
    \xi = &\cos(k\Delta x) - \frac{v\Delta t}{ \Delta x} i\sin(k\Delta x) \nonumber 

\end{align}

<span>

<span style="color:#78AE7E">

<!-- We know that the von Neumann stability condition $|\xi(k)|^2 \leq 1$ holds for every possible value of $k$. T  -->

We insert our expression for $\xi$ into the von Neumann stability condition $|\xi(k)|^2 \leq 1$ and get

\begin{align}
    \left| \cos(k\Delta x) - \frac{v\Delta t}{\Delta x} i \sin(k\Delta x) \right|^2 \leq 1 \nonumber
\end{align}

In order for this condition to hold for all possible values of $k$, we see that $(\frac{v\Delta t}{\Delta x})^2 \leq 1$. This gives us the CFL condition: 

\begin{equation}
    \xi \leq \frac{v\Delta t}{\Delta x} \nonumber
\end{equation}

<span>

Add this to your library and solve the previous simulation imposing the CFL condition. For this exercise, fill in `nm_lib` the function `evolv_Lax_uadv_burgers`. 

In [4]:
# Define the domain 
x0 = -2.6
xf = 2.6

nx = 128
nt = 50
xx = np.linspace(x0, xf, nx) 

t, unnt = nm.evolv_Lax_uadv_burgers(xx, u_0(xx), nt, ddx=nm.deriv_dnw, bnd_limits=[0, 1])

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 3))
anim = FuncAnimation(fig, animate, interval=20, frames=nt, init_func=init)
plt.close(); HTML(anim.to_jshtml())

/Users/juanms/Numerical_sim_course/students_2023/master/Aline_Rangoy_Brunvoll/nm_lib/nm_lib/nm_lib.py:135: RuntimeWarning: divide by zero encountered in divide
  return np.min(dx / np.abs(a))


## 1- Diffusive. 

Redo the exercise [ex_2b](https://github.com/AST-Course/AST5110/blob/main/ex_2b.ipynb) and compare the two methods, i.e., one from 
[ex_2b](https://github.com/AST-Course/AST5110/blob/main/ex_2b.ipynb) and the Lax-Method. 

Which one is more diffusive? Why? Rewrite Lax-method, so the right-hand side is as follows: 

$$\frac{u^{n+1}-u^{n}}{\Delta t} = ...$$

What is the reminder term look like? Is the order of convergence the same for the two methods? 

<span style="color:#78AE7E">

The Lax timestep method is given as 

\begin{equation*}
u^{n+1}_j = \frac{1}{2}(u^n_{j+1} + u^n_{j-1}) - \frac{v \Delta t}{2 \Delta x}(u^n_{j+1} - u^n_{j-1}).
\end{equation*} 

We can rewrite this as indicated in the problem text. 

$$ \frac{u^{n+1} - u^n }{\Delta t} = \frac{1}{2\Delta t}(u^n_{j+1} + u^n_{j-1}) - \frac{v}{2 \Delta x}(u^n_{j+1} - u^n_{j-1}) - \frac{1}{\Delta t} u_j^n $$

<span>

<span style="color:#78AE7E">

This again can be expressed as a differential equation

\begin{equation*}
    \frac{\partial u}{\partial t} = - v\frac{\partial u}{\partial x} + \frac{(\Delta x)^2}{2\Delta t} \nabla^2 u.
\end{equation*}

<span>

<span style="color:#78AE7E">

Here the remainder term (the last term on the right-hand side) is a diffusive term, and the equation has a 2nd order of convergence.

<span>

In [5]:
init_2 = lambda: axes.plot(xx, unnt[:, 0]); axes.plot(xx, unnt[:, 0])

def animate_2(i):
    """Animate the solution with dynamic axis limits"""
    axes.clear()
    axes.plot(xx, unnt[:, i], label='Lax')
    axes.plot(xx, unnt_2[:, i], label='Upwind', ls='--')
    axes.legend()
    axes.set_title('t=%.2f'%t[i])

t, unnt = nm.evolv_Lax_uadv_burgers(xx, u_0(xx), nt, cfl_cut=0.98, ddx=nm.deriv_upw, bnd_limits=[1, 0])
t_2, unnt_2 = nm.evolv_uadv_burgers(xx, u_0(xx), nt, cfl_cut=0.98, a=1, ddx=nm.deriv_upw, bnd_limits=[1, 0])

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 3))
anim = FuncAnimation(fig, animate_2, interval=50, frames=nt, init_func=init_2)
plt.close()
HTML(anim.to_jshtml())

/Users/juanms/Numerical_sim_course/students_2023/master/Aline_Rangoy_Brunvoll/nm_lib/nm_lib/nm_lib.py:135: RuntimeWarning: divide by zero encountered in divide
  return np.min(dx / np.abs(a))


<span style="color:#78AE7E">

We see that the upwind scheme is less diffusive than the LAX method. The upwind scheme accurately represents the sharp shock as it travels.

However, it seems the upwind solution has a phase error which makes it lag behind the Lax method, and presumably the shock itself.


<span>

<span style="color:pink"> JMS, ok, now I'm pink, blue, red, yellow and orange :-). </span>.

<span style="color:blue"> Good! Towards the end the upwind has a different problem which has nothing to do with diffusion but instead it is due to being not conservative.</span>.


---

(\*) Equation (1) is, in fact, a shortened version of the full Burgers’ equation, which contains a viscosity term on the right-hand side, as follows:
$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}.$$